# Optimize inputs given output

- Find inputs by optimizing from the learned forward model with given output.
    - 1) By randomizing the inputs multiple times, get the optimized inputs
    - 2) Get a single input instance, move along the null space by the small amount, compute the new null space and move along the line  (repeat)

2019-07-27

In [30]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns

import ipdb as pdb
import os
import re
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from time import time
from itertools import product
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
print(tf.__version__)

tf.compat.v1.random.set_random_seed(0)
np.random.seed(0)

In [30]:
# Disable eager execution
tf.compat.v1.disable_eager_execution()

2.0.0-beta1


## Settings

In [31]:
#####################################################
target = np.array([[300, 1800]]) # y target (F1, F2)
#####################################################

spkr_data = 'JW12.pckl'
n_components = 3  # PCA
input_dim = n_components
output_dim = 2

BATCH_SIZE = 10
TEST_SIZE = 0.2  # 0-1
MAX_EPOCH = 100
n_display = 10 # log every 10th
n_patience = 30 # for early stopping

model_file = 'forward_model.h5'

In [32]:
# Load speaker data
with open(spkr_data, 'rb') as f:
    SPKR = pickle.load(f)
    
SPKR.head()

,Subj,FName,PreLab,NexLab,Label,Context,Dur,T1x,T1y,T2x,...,segEnd,GONS,NONS,MAXC,NOFFS,GOFFS,GesTarDur,timeMidWithinGest,timeMid2GestDiff,GestFound
0,JW12,JW12_TP007,P,N,AE1,P-AE1-N,0.119728,-15.086,-1.4980,-29.573500,...,5.962424,5.801770,5.884162,5.904760,5.918492,5.966554,0.034330,1.0,0.000000,1
1,JW12,JW12_TP010,HH,D,AE1,HH-AE1-D,0.099773,-17.583,3.0545,-30.327500,...,6.339805,6.076410,6.186266,6.206864,6.241194,6.378514,0.054928,0.0,0.048724,1
2,JW12,JW12_TP017,N,S,AE1,N-AE1-S,0.119728,-10.927,-0.0570,-25.707001,...,1.635277,1.380066,1.510520,1.592912,1.613510,1.682170,0.102990,1.0,0.000000,1
3,JW12,JW12_TP017,K,R,AE1,K-AE1-R,0.079819,-22.037,12.3780,-33.766001,...,2.283803,2.100996,2.300110,2.320708,2.348172,2.444296,0.048062,0.0,0.056216,1
4,JW12,JW12_TP017,IY0,N,AE1,IY0-AE1-N,0.049887,-15.916,7.5980,-29.587999,...,2.583123,2.492358,2.602214,2.615946,2.636544,2.705204,0.034330,0.0,0.044035,1


In [33]:
# Load model
model = tf.keras.models.load_model(model_file)

model.summary()

W0727 23:53:52.976483 4466156992 backend.py:527] OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                40        
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 172
Trainable params: 172
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.layers

## Build model

In [43]:
input_var = K.random_normal_variable((1, input_dim), 0, 1)
input_tensor = Input(tensor=input_var)

layer1 = model.layers[0]
layer1.trainable = False
layer2 = model.layers[2]
layer2.trainable = False
layer3 = model.layers[4]
layer3.trainable = False

x = layer1(input_tensor)
x = layer2(x)
x = layer3(x)

y = tf.constant(target)

inv_model = Model(input_tensor, y)

In [46]:
def custom_loss(y, ypred):
    return K.mean(K.square(ypred - y), axis=-1)

In [47]:
model.compile(optimizer='rmsprop',
              loss=custom_loss,
              metrics=['accuracy'])

In [50]:
model.fit(input_var, y, steps_per_epoch=10)

Train on 10 samples
10/10 [==============================] - 0s 9ms/step - loss: 1665772.5250 - accuracy: 0.0000e+00


# 모델 다시 만들기. 인풋을 넣는 과정부터 다시 생각해보기.